In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Reference
https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
# extract the whole website
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

In [4]:
# extract the whole table
My_table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
# extract all things about postcodes, borough and neighborhood
lists = My_table.findAll('td')
newlists = []
for item in lists:
    tmpstr = str(item)
    if tmpstr.find('</a>') == -1:
        newlists.append(tmpstr[tmpstr.find('<td>')+4:tmpstr.find('</td>')].replace('\n',''))
    else:
        newlists.append(tmpstr[tmpstr.find('">')+2:tmpstr.find('</a>')].replace('\n',''))
        
newlists[:6]

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned']

In [6]:
# extract columns
postcodes = newlists[0::3]
borough = newlists[1::3]
neighborhood = newlists[2::3]

In [7]:
neighborhood[-3:]

['Royal York South West', 'South of Bloor', 'Not assigned']

#### Solve:
#### Ignore cells with a borough that is Not assigned.
#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 


In [8]:
p,b,n = [],[],[]
for i in range(len(postcodes)):
    if borough[i] != 'Not assigned':
        p.append(postcodes[i])
        b.append(borough[i])
        if neighborhood[i] == 'Not assigned':
            n.append(borough[i])
        else:
            n.append(neighborhood[i])

In [9]:
from operator import itemgetter

#### Solve:
#### Combine Multiple rows with the same postcodes and boroughs to one row of neighborhood

In [14]:
POST,BOR,NEIGH = [],[],[]
for i in range(len(p)):
    if p[i] not in POST:
        POST.append(p[i])
        BOR.append(b[i])
        tmpid = [j for j,x in enumerate(p) if x==p[i]]
        tmgetter = itemgetter(*tmpid)
        tmplist = tmgetter(n)
        if isinstance(tmplist,str):
            NEIGH.append(tmplist)
        elif isinstance(tmplist,tuple):
            NEIGH.append(','.join(list(tmplist)))

In [15]:
df = pd.DataFrame()
df['PostalCode'] = POST
df['Borough'] = BOR
df['Neighborhood'] = NEIGH

#### Whole dataframe

In [17]:
df

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A      Queen's Park   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   
15         M5C  Downtown Toronto   
16         M6C              York   
17         M9C         Etobicoke   
18         M1E       Scarborough   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
21         M6E              York   
22         M1G       Scarborough   
23         M4G         East York   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
26         M1H       Scarborough   
27         M2H        North York   
28         M3H        North York   
29         M4H         East York   
30         M5H  Downtown Toronto   
31         M6H      West Toronto   
32         M1J       Scarborough   
33         M2J        North York   
34         M3J        North York   
35         M4J         East York   
36         M5J  Downtown Toronto   
37         M6J      West Toronto   
38         M1K       Scarborough   
39         M2K        North York   
40         M3K        North York   
41         M4K      East Toronto   
42         M5K  Downtown Toronto   
43         M6K      West Toronto   
44         M1L       Scarborough   
45         M2L        North York   
46         M3L        North York   
47         M4L      East Toronto   
48         M5L  Downtown Toronto   
49         M6L        North York   
50         M9L        North York   
51         M1M       Scarborough   
52         M2M        North York   
53         M3M        North York   
54         M4M      East Toronto   
55         M5M        North York   
56         M6M              York   
57         M9M        North York   
58         M1N       Scarborough   
59         M2N        North York   
60         M3N        North York   
61         M4N   Central Toronto   
62         M5N   Central Toronto   
63         M6N              York   
64         M9N              York   
65         M1P       Scarborough   
66         M2P        North York   
67         M4P   Central Toronto   
68         M5P   Central Toronto   
69         M6P      West Toronto   
70         M9P         Etobicoke   
71         M1R       Scarborough   
72         M2R        North York   
73         M4R   Central Toronto   
74         M5R   Central Toronto   
75         M6R      West Toronto   
76         M7R       Mississauga   
77         M9R         Etobicoke   
78         M1S       Scarborough   
79         M4S   Central Toronto   
80         M5S  Downtown Toronto   
81         M6S      West Toronto   
82         M1T       Scarborough   
83         M4T   Central Toronto   
84         M5T  Downtown Toronto   
85         M1V       Scarborough   
86         M4V   Central Toronto   
87         M5V  Downtown Toronto   
88         M8V         Etobicoke   
89         M9V         Etobicoke   
90         M1W       Scarborough   
91         M4W  Downtown Toronto   
92         M5W  Downtown Toronto   
93         M8W         Etobicoke   
94         M9W         Etobicoke   
95         M1X       Scarborough   
96         M4X  Downtown Toronto   
97         M5X  Downtown Toronto   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                             Harbourfront,Regent Park  
3                      Lawrence

#### Show the same content from the instruction

In [22]:
df.loc[df['PostalCode'].isin(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]

PostalCode           Borough  \
2         M5A  Downtown Toronto   
6         M1B       Scarborough   
8         M4B         East York   
23        M4G         East York   
24        M5G  Downtown Toronto   
27        M2H        North York   
32        M1J       Scarborough   
50        M9L        North York   
54        M4M      East Toronto   
71        M1R       Scarborough   
87        M5V  Downtown Toronto   
89        M9V         Etobicoke   

                                         Neighborhood  
2                            Harbourfront,Regent Park  
6                                       Rouge,Malvern  
8                      Woodbine Gardens,Parkview Hill  
23                                            Leaside  
24                                 Central Bay Street  
27                                  Hillcrest Village  
32                                Scarborough Village  
50                                      Humber Summit  
54                                    Studio District  
71                                   Maryvale,Wexford  
87  CN Tower,Bathurst Quay,Island airport,Harbourf...  
89  Albion Gardens,Beaumond Heights,Humbergate,Jam...

#### Dataframe shape

In [18]:
df.shape

(103, 3)